In [5]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from datetime import datetime, timedelta

In [15]:
def generator():
    while True:
        yield

In [35]:
from datetime import timedelta, date

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

start_dt = date(2005, 1, 1)
end_dt = date(2023, 6, 30)
dates = []
for dt in daterange(start_dt, end_dt):
    dates.append(dt.strftime("%Y-%m-%d"))
    
dates = dates[::-1]
len(dates)

6755

In [36]:
def crawl(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a', {'rel': "bookmark"})
    a = [a_.get('href') for a_ in a if len(a_.get('href'))]
    return a

In [41]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

data = []
for i in tqdm(range(0, len(dates), max_worker)):
    b = dates[i: i + max_worker]
    urls = [f'http://blog.malaysia-asia.my/search?updated-max={t}T14:15:00%2B08:00&max-results=7&start=4&by-date=false' for t in b]
    done = False
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, url): url for url in urls}
        
        for future in as_completed(futures):
            r = future.result()
            data.extend(r)
            if not len(r):
                done = True
    
    if done:
        break

 23%|█████████▍                               | 155/676 [02:23<08:03,  1.08it/s]


In [42]:
len(set(data))

176

In [44]:
with open('url.json', 'w') as fopen:
    json.dump(list(set(data)), fopen)